<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/Lung_Cancer_Multiclassification_Model_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install important libraries.
!pip install opendatasets
!pip install scikeras

In [2]:
# Import necessary libraries and functions.
import opendatasets as od
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [ ]:
# Download a dataset using opendatasets.
od.download("https://www.kaggle.com/datasets/bhaveshmisra/lung-cancer-images12000-imagesmostly")

In [4]:
# Create an image data generator for data augmentation.
datagen = ImageDataGenerator(rescale=1.0/255.0, rotation_range=20, width_shift_range=0.2,
                            height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                            horizontal_flip=True, vertical_flip=True, channel_shift_range=0.1,
                            fill_mode='nearest')

In [ ]:
# Split data into training and test sets.
train_iterator = datagen.flow_from_directory("lung-cancer-images12000-imagesmostly/dataset/training_set",
                                            class_mode="categorical", batch_size=200)
test_iterator = datagen.flow_from_directory("lung-cancer-images12000-imagesmostly/dataset/test_set",
                                           class_mode="categorical", batch_size=30)

In [6]:
# Create a CNN model.
def create_model():
    model = Sequential()
    model.add(Conv2D(12, (3, 3), padding="same", input_shape=(256, 256, 3)))
    model.add(Conv2D(4, (4, 4), strides=(2, 2), padding="same", activation="relu"))
    model.add(MaxPooling2D())
    model.add(Conv2D(13, (2, 3)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    return model


In [ ]:
# Train the model with the best validation parameters.
model = create_model()
model.fit(train_iterator, validation_data=test_iterator,epochs=12,batch_size=32)

In [ ]:
# Serialize model architecture to JSON.
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# Serialize model weights to HDF5.
model.save_weights("model_weights.h5")